In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
#conf.set('spark.sql.repl.eagerEval.enabled', True)

conf.set('spark.driver.memory','64g')


sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 16:03:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from pyspark.ml.feature import CountVectorizer, Tokenizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import Normalizer
from pyspark.sql.functions import col
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType
from pyspark.sql.functions import from_unixtime, to_date

# Define the schema for the reviews dataset without image data
# review_schema = StructType([
#     StructField("rating", FloatType()),
#     #StructField("title", StringType()),
#     #StructField("text", StringType()),
#     StructField("asin", FloatType()),
#     #StructField("parent_asin", FloatType()),
#     StructField("user_id", FloatType()),
#     #StructField("timestamp", IntegerType()),
#     #StructField("verified_purchase", BooleanType()),
#     #StructField("helpful_vote", IntegerType())
# ])

In [ ]:
#reviews_df = spark.read.json("Amazon_Fashion.jsonl",schema = review_schema)

In [ ]:
# Load metadata; assume no complex schema is necessary for metadata
from pyspark.sql.types import *

meta_schema = StructType([
    StructField("parent_asin", StringType(), True),  # Amazon Standard Identification Number
    StructField("title", StringType(), True),  # Title of the book
    StructField("price", FloatType(), True),  # Price of the book
    StructField("description", ArrayType(StringType()), True),  # Description of the book
    StructField("main_category", StringType(), True),  # Parent ASIN if this is a variation of a product
    StructField("features", StringType(), True),
    StructField("categories", ArrayType(StringType()), True)
])

In [ ]:
# Load user reviews with the defined schema
meta_df = spark.read.schema(meta_schema).json("meta_Amazon_Fashion.jsonl")



In [ ]:
meta_df.printSchema()

root
 |-- parent_asin: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: float (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- main_category: string (nullable = true)
 |-- features: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
meta_df.limit(10).toPandas()

,parent_asin,title,price,description,main_category,features,categories
0,B08BHN9PK5,YUEDGE 5 Pairs Men's Moisture Control Cushione...,NaN,[],AMAZON FASHION,[],[]
1,B08R39MRDW,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,NaN,[],AMAZON FASHION,"[""Drawstring closure"",""Machine Wash""]",[]
2,B077KJHCJ4,Pastel by Vivienne Honey Vanilla Girls' Trapez...,NaN,[],AMAZON FASHION,"[""Zipper closure"",""Hand Wash Only""]",[]
3,B0811M2JG9,Mento Streamtail,29.809999,[Slip on the Women's Mento and you're ready to...,AMAZON FASHION,"[""Thermoplastic Rubber sole"",""High Density Pre...",[]
4,B07SB2892S,RONNOX Women's 3-Pairs Bright Colored Calf Com...,17.990000,[Ronnox Calf Sleeves - Allowing Your Body to P...,AMAZON FASHION,"[""Pull On closure"",""Size Guide: \""S\"" fits cal...",[]
5,B07NZ7V22C,12pairs Egowz High Visibility Nylon Latex Foam...,NaN,[],AMAZON FASHION,[],[]
6,B01ISW76HQ,Nemidor Women's Vintage 1950s Style Sleeved Pl...,NaN,[],AMAZON FASHION,[],[]
7,B08T6KT3N8,YUNXI 3D grape Drop Earrings Cute Fruit Gold D...,NaN,[],AMAZON FASHION,[],[]
8,B08FMLXY1Z,LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...,9.990000,[Tie dye shirts for Women long sleeve crewneck...,AMAZON FASHION,"[""Pull On closure""]",[]
9,B07DXHY5ZX,PattyBoutik Women Crewneck Eyelet Cold Shoulde...,NaN,[],AMAZON FASHION,"[""97% Cotton, 3% Other Fibers""]",[]


In [ ]:
# Select relevant columns from metadata
meta_df = meta_df.select( "parent_asin", "title", "description", "price", "categories", "features")

# Combine relevant text columns into one for feature extraction
meta_df = meta_df.withColumn("text", concat_ws(" ", "title", "categories", "description", "features"))

# Tokenize text
tokenizer = Tokenizer(inputCol="text", outputCol="words")
meta_df = tokenizer.transform(meta_df)

In [ ]:
meta_df.limit(10).toPandas()

,parent_asin,title,description,price,categories,features,text,words
0,B08BHN9PK5,YUEDGE 5 Pairs Men's Moisture Control Cushione...,[],NaN,[],[],YUEDGE 5 Pairs Men's Moisture Control Cushione...,"[yuedge, 5, pairs, men's, moisture, control, c..."
1,B08R39MRDW,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,[],NaN,[],"[""Drawstring closure"",""Machine Wash""]",DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,"[doubcq, women's, palazzo, lounge, wide, leg, ..."
2,B077KJHCJ4,Pastel by Vivienne Honey Vanilla Girls' Trapez...,[],NaN,[],"[""Zipper closure"",""Hand Wash Only""]",Pastel by Vivienne Honey Vanilla Girls' Trapez...,"[pastel, by, vivienne, honey, vanilla, girls',..."
3,B0811M2JG9,Mento Streamtail,[Slip on the Women's Mento and you're ready to...,29.809999,[],"[""Thermoplastic Rubber sole"",""High Density Pre...",Mento Streamtail Slip on the Women's Mento and...,"[mento, streamtail, slip, on, the, women's, me..."
4,B07SB2892S,RONNOX Women's 3-Pairs Bright Colored Calf Com...,[Ronnox Calf Sleeves - Allowing Your Body to P...,17.990000,[],"[""Pull On closure"",""Size Guide: \""S\"" fits cal...",RONNOX Women's 3-Pairs Bright Colored Calf Com...,"[ronnox, women's, 3-pairs, bright, colored, ca..."
5,B07NZ7V22C,12pairs Egowz High Visibility Nylon Latex Foam...,[],NaN,[],[],12pairs Egowz High Visibility Nylon Latex Foam...,"[12pairs, egowz, high, visibility, nylon, late..."
6,B01ISW76HQ,Nemidor Women's Vintage 1950s Style Sleeved Pl...,[],NaN,[],[],Nemidor Women's Vintage 1950s Style Sleeved Pl...,"[nemidor, women's, vintage, 1950s, style, slee..."
7,B08T6KT3N8,YUNXI 3D grape Drop Earrings Cute Fruit Gold D...,[],NaN,[],[],YUNXI 3D grape Drop Earrings Cute Fruit Gold D...,"[yunxi, 3d, grape, drop, earrings, cute, fruit..."
8,B08FMLXY1Z,LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...,[Tie dye shirts for Women long sleeve crewneck...,9.990000,[],"[""Pull On closure""]",LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...,"[lyckyy, women's, tie, dye, sweatshirt, crewne..."
9,B07DXHY5ZX,PattyBoutik Women Crewneck Eyelet Cold Shoulde...,[],NaN,[],"[""97% Cotton, 3% Other Fibers""]",PattyBoutik Women Crewneck Eyelet Cold Shoulde...,"[pattyboutik, women, crewneck, eyelet, cold, s..."


In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
meta_df = remover.transform(meta_df)

# Convert words into numerical features
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures")

pipeline = Pipeline(stages=[hashingTF, idf])
model = pipeline.fit(meta_df)
meta_df = model.transform(meta_df)

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
meta_df.limit(3).toPandas()

,parent_asin,title,description,price,categories,features,text,words,filtered_words,rawFeatures,idfFeatures
0,B08BHN9PK5,"YUEDGE 5 Pairs Men's Moisture Control Cushioned Dry Fit Casual Athletic Crew Socks for Men (Blue, Size 9-12)",[],NaN,[],[],"YUEDGE 5 Pairs Men's Moisture Control Cushioned Dry Fit Casual Athletic Crew Socks for Men (Blue, Size 9-12) []","[yuedge, 5, pairs, men's, moisture, control, cushioned, dry, fit, casual, athletic, crew, socks, for, men, (blue,, size, 9-12), []]","[yuedge, 5, pairs, men's, moisture, control, cushioned, dry, fit, casual, athletic, crew, socks, men, (blue,, size, 9-12), []]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.6081998762804752, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
1,B08R39MRDW,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Flower Mix Blue, XL)",[],NaN,[],"[""Drawstring closure"",""Machine Wash""]","DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Flower Mix Blue, XL) [""Drawstring closure"",""Machine Wash""]","[doubcq, women's, palazzo, lounge, wide, leg, casual, flowy, pants(flower, mix, blue,, xl), [""drawstring, closure"",""machine, wash""]]","[doubcq, women's, palazzo, lounge, wide, leg, casual, flowy, pants(flower, mix, blue,, xl), [""drawstring, closure"",""machine, wash""]]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2430100929154024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
2,B077KJHCJ4,Pastel by Vivienne Honey Vanilla Girls' Trapeze Dress with Easy Removable Label Large 9-10 Years Navy,[],NaN,[],"[""Zipper closure"",""Hand Wash Only""]","Pastel by Vivienne Honey Vanilla Girls' Trapeze Dress with Easy Removable Label Large 9-10 Years Navy [""Zipper closure"",""Hand Wash Only""]","[pastel, by, vivienne, honey, vanilla, girls', trapeze, dress, with, easy, removable, label, large, 9-10, years, navy, [""zipper, closure"",""hand, wash, only""]]","[pastel, vivienne, honey, vanilla, girls', trapeze, dress, easy, removable, label, large, 9-10, years, navy, [""zipper, closure"",""hand, wash, only""]]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [ ]:
# Select necessary columns
meta_df = meta_df.select("title", "parent_asin", "idfFeatures")

# Assemble feature vectors
assembler = VectorAssembler(inputCols=["idfFeatures"], outputCol="features_vec")
meta_df = assembler.transform(meta_df)

# Normalize feature vectors
normalizer = Normalizer(inputCol="features_vec", outputCol="normalized_features")
meta_df = normalizer.transform(meta_df)


In [ ]:
# # Select relevant columns from reviews data
# reviews_df = reviews_df.select("user_id", "asin", "rating")

# # Join reviews data with metadata
# joined_df = reviews_df.join(meta_df, "asin")

# # Create a utility matrix where rows represent users, columns represent items, and cells represent ratings
# utility_matrix = joined_df.groupby("user_id", "asin", "features_vec").avg("rating").fillna(0)

In [ ]:
#joined_df.printSchema()

In [ ]:
#utility_matrix.printSchema()

In [ ]:
meta_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- idfFeatures: vector (nullable = true)
 |-- features_vec: vector (nullable = true)
 |-- normalized_features: vector (nullable = true)



In [ ]:
# Compute item-item similarity using cosine similarity
from pyspark.ml.feature import BucketedRandomProjectionLSH

bucketedRandomProjectionLSH = BucketedRandomProjectionLSH(inputCol="normalized_features", outputCol="hashes", bucketLength=0.5)
model = bucketedRandomProjectionLSH.fit(meta_df)
similar_items = model.approxSimilarityJoin(meta_df, meta_df, 5.0, distCol="similarity")

from pyspark.sql.functions import col, expr

def get_recommendations(item_id, num_recommendations=10):
    # Filter similar items for the specified item_id
    similar_items_filtered = similar_items.filter(col("datasetA.parent_asin") == item_id)

    # Convert distance to similarity score
    similar_items_filtered = similar_items_filtered.withColumn("similarity", expr("1 / (1 + similarity)"))

    # Join with metadata to get the titles of recommended items
    recommendations = similar_items_filtered.join(meta_df, col("datasetB.parent_asin") == meta_df.parent_asin) \
                                            .select(
                                                col("datasetA.parent_asin").alias("original_item"),
                                                col("datasetB.parent_asin").alias("recommended_item"),
                                                col("title").alias("recommended_item_title"),
                                                col("similarity")
                                            ) \
                                            .orderBy("similarity", ascending=False) \
                                            .limit(num_recommendations)
    return recommendations




In [ ]:
# Example usage
item_id = "B0811M2JG9"  # Example item ID
recommendations = get_recommendations(item_id)
recommendations.show(truncate=False)


+-------------+----------------+---------------------------------------------------------------------------------------------------------------------------+-------------------+
|original_item|recommended_item|recommended_item_title                                                                                                     |similarity         |
+-------------+----------------+---------------------------------------------------------------------------------------------------------------------------+-------------------+
|B0811M2JG9   |B0811M2JG9      |Mento Streamtail                                                                                                           |1.0                |
|B0811M2JG9   |B08MQ28DMK      |FEOYA Panties for Women High Waist Seamless Briefs Panties Easy Clean Lightweight Breathable Ladies Panties Multicolor Pink|0.44894537153168707|
|B0811M2JG9   |B08WJJFDRD      |Guy Harvey Boy's Long Sleeve Performance Shirt with 30+ UPF Sun Protection         

In [ ]:
recommendations.show()

+-------------+----------------+----------------------+-------------------+
|original_item|recommended_item|recommended_item_title|         similarity|
+-------------+----------------+----------------------+-------------------+
|   B0811M2JG9|      B0811M2JG9|      Mento Streamtail|                1.0|
|   B0811M2JG9|      B08MQ28DMK|  FEOYA Panties for...|0.44894537153168707|
|   B0811M2JG9|      B08WJJFDRD|  Guy Harvey Boy's ...|0.44761148404411905|
|   B0811M2JG9|      B01CYSBK86|  Women's Sport X-T...| 0.4469941780036802|
|   B0811M2JG9|      B00UN6FTR0|  Men's Sperry, Pen...| 0.4460840640856478|
|   B0811M2JG9|      B00I2YTKQG|               temp490|0.44589727527050144|
|   B0811M2JG9|      B00NSDK8K8|  Hanes Women`s Con...|  0.445792235862771|
|   B0811M2JG9|      B01MYN08B2|  Nabtos Women's Un...| 0.4445766774203279|
|   B0811M2JG9|      B0021FFJ44|          Smartest One|0.44428230355141496|
|   B0811M2JG9|      B06ZYY96MX|  Hanes Women's X-T...| 0.4438135039256352|
+-----------

In [ ]:
# Example usage
item_id = "B08R39MRDW"
recommendations = get_recommendations(item_id)
recommendations.show()

+-------------+----------------+----------------------+------------------+
|original_item|recommended_item|recommended_item_title|        similarity|
+-------------+----------------+----------------------+------------------+
|   B08R39MRDW|      B08R39MRDW|  DouBCQ Women's Pa...|               1.0|
|   B08R39MRDW|      B08R39X39B|  DouBCQ Women's Pa...|0.6019333249445532|
|   B08R39MRDW|      B08R3BZKQ7|  DouBCQ Women's Pa...|0.5987252325216763|
|   B08R39MRDW|      B08R39RB23|  DouBCQ Women's Pa...|0.5886052206353717|
|   B08R39MRDW|      B085CFGM1P|  DouBCQ Women's Su...|0.5634587036749223|
|   B08R39MRDW|      B08R3CTLRY|  DouBCQ Women's Pa...| 0.559458397336712|
|   B08R39MRDW|      B08R3BMFQY|  DouBCQ Women's Pa...| 0.554099054842797|
|   B08R39MRDW|      B08R39Z847|  DouBCQ Women's Pa...|0.5480119797307619|
|   B08R39MRDW|      B08R38J99Z|  DouBCQ Women's Pa...|0.5478404229627479|
|   B08R39MRDW|      B085CFKBCK|  DouBCQ Women's Su...|0.5305934868767245|
+-------------+----------

In [ ]:
item_id = "B0811M2JG9"
recommendations = get_recommendations(item_id)
recommendations.show()

+-------------+----------------+----------------------+-------------------+
|original_item|recommended_item|recommended_item_title|         similarity|
+-------------+----------------+----------------------+-------------------+
|   B0811M2JG9|      B0811M2JG9|      Mento Streamtail|                1.0|
|   B0811M2JG9|      B08MQ28DMK|  FEOYA Panties for...|0.44894537153168707|
|   B0811M2JG9|      B08WJJFDRD|  Guy Harvey Boy's ...|0.44761148404411905|
|   B0811M2JG9|      B01CYSBK86|  Women's Sport X-T...| 0.4469941780036802|
|   B0811M2JG9|      B00UN6FTR0|  Men's Sperry, Pen...| 0.4460840640856478|
|   B0811M2JG9|      B00I2YTKQG|               temp490|0.44589727527050144|
|   B0811M2JG9|      B00NSDK8K8|  Hanes Women`s Con...|  0.445792235862771|
|   B0811M2JG9|      B01MYN08B2|  Nabtos Women's Un...| 0.4445766774203279|
|   B0811M2JG9|      B0021FFJ44|          Smartest One|0.44428230355141496|
|   B0811M2JG9|      B06ZYY96MX|  Hanes Women's X-T...| 0.4438135039256352|
+-----------

In [ ]:
# Pivot the DataFrame to form a similarity matrix
similar_items_filtered = similar_items_filtered.withColumn("similarity", expr("1 / (1 + similarity)"))

In [ ]:
similar_items_filtered.head()

Row(datasetA=Row(title='Mento Streamtail', parent_asin='B0811M2JG9', idfFeatures=SparseVector(2000, {13: 6.2166, 17: 5.1754, 65: 3.6723, 76: 5.7972, 78: 4.8819, 94: 11.4219, 99: 3.0269, 116: 2.6874, 173: 8.0287, 178: 4.4587, 252: 4.6494, 255: 11.5472, 322: 4.7121, 336: 4.3509, 351: 3.1744, 415: 2.887, 502: 4.793, 546: 4.516, 557: 4.8181, 558: 8.3613, 616: 10.9637, 691: 5.6803, 767: 10.5921, 791: 5.2505, 896: 12.5085, 897: 3.8204, 933: 1.524, 956: 3.7286, 1011: 4.6803, 1022: 3.1198, 1128: 8.1411, 1233: 4.3223, 1265: 3.0316, 1381: 4.6367, 1455: 4.4862, 1463: 4.3709, 1473: 5.0083, 1501: 9.3682, 1506: 7.0264, 1538: 8.805, 1599: 7.9602, 1665: 4.4689, 1673: 5.2607, 1674: 4.4827, 1755: 4.3796, 1769: 10.9596, 1796: 4.5601, 1825: 5.22, 1832: 5.9678, 1862: 5.2229, 1903: 10.2379}), features_vec=SparseVector(2000, {13: 6.2166, 17: 5.1754, 65: 3.6723, 76: 5.7972, 78: 4.8819, 94: 11.4219, 99: 3.0269, 116: 2.6874, 173: 8.0287, 178: 4.4587, 252: 4.6494, 255: 11.5472, 322: 4.7121, 336: 4.3509, 351: 3.1

In [ ]:
# The following is for final combined analysis
# pick top 10 'parent_asin'

item_id = "B08R39MRDW"
recommendations = get_recommendations(item_id)
recommendations.show(10)

+-------------+----------------+----------------------+------------------+
|original_item|recommended_item|recommended_item_title|        similarity|
+-------------+----------------+----------------------+------------------+
|   B08R39MRDW|      B08R39MRDW|  DouBCQ Women's Pa...|               1.0|
|   B08R39MRDW|      B08R39X39B|  DouBCQ Women's Pa...|0.6019333249445532|
|   B08R39MRDW|      B08R3BZKQ7|  DouBCQ Women's Pa...|0.5987252325216763|
|   B08R39MRDW|      B08R39RB23|  DouBCQ Women's Pa...|0.5886052206353717|
|   B08R39MRDW|      B085CFGM1P|  DouBCQ Women's Su...|0.5634587036749223|
|   B08R39MRDW|      B08R3CTLRY|  DouBCQ Women's Pa...| 0.559458397336712|
|   B08R39MRDW|      B08R3BMFQY|  DouBCQ Women's Pa...| 0.554099054842797|
|   B08R39MRDW|      B08R39Z847|  DouBCQ Women's Pa...|0.5480119797307619|
|   B08R39MRDW|      B08R38J99Z|  DouBCQ Women's Pa...|0.5478404229627479|
|   B08R39MRDW|      B085CFKBCK|  DouBCQ Women's Su...|0.5305934868767245|
+-------------+----------

In [ ]:
recommendations.count()

10

In [ ]:
recommendations.printSchema()

root
 |-- original_item: string (nullable = true)
 |-- recommended_item: string (nullable = true)
 |-- recommended_item_title: string (nullable = true)
 |-- similarity: double (nullable = true)



In [ ]:
df_reviews = spark\
    .read\
    .format("json")\
    .load("Amazon_Fashion.jsonl")\
    .repartition(20)
df_reviews.rdd.getNumPartitions()
df_reviews.count()

2500939

In [ ]:
df_reviews = df_reviews.dropDuplicates(['asin', 'user_id', 'timestamp', 'text'])
df_reviews.count()

2475694

In [ ]:
# remove duplicate reviews due to user_id suffix
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, row_number, expr

# Define the window specification
windowSpec = Window.partitionBy("text", "timestamp").orderBy(expr("user_id LIKE '%\_%'"))

# Use row_number to assign a rank to each row within each group, prioritized by user_id without '_'
ranked_df = df_reviews.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the rows ranked 1, i.e., top priority row per group
df_reviews = ranked_df.filter(col("rank") == 1).drop("rank")

df_reviews.count()

2475451

In [ ]:
# remove the original item from 'recommendations'
input_df = recommendations.filter(recommendations.similarity != 1.0)
input_df.count()

9

In [ ]:
input_df = input_df.select("recommended_item") # drop irrelevant columns
input_df.cache()
input_df.show(10)

+----------------+
|recommended_item|
+----------------+
|      B08R39X39B|
|      B08R3BZKQ7|
|      B08R39RB23|
|      B085CFGM1P|
|      B08R3CTLRY|
|      B08R3BMFQY|
|      B08R39Z847|
|      B08R38J99Z|
|      B085CFKBCK|
+----------------+



In [ ]:
from pyspark.sql.functions import col, avg, count, sum

# Join the DataFrames on parent_asin
joined_df = input_df.join(df_reviews, input_df.recommended_item == df_reviews.parent_asin)

# Group by parent_asin and calculate average rating, count of ratings, and sum of helpful votes
average_ratings = joined_df.groupBy("parent_asin").agg(
    avg("rating").alias("average_rating"),
    sum("helpful_vote").alias("total_helpful_votes"),
    count("rating").alias("rating_count")
)

# Order by average rating, rating count, and helpful votes in descending order
top_parent_asins = average_ratings.orderBy(
    col("average_rating").desc(),
    col("total_helpful_votes").desc(),
    col("rating_count").desc()
)

# Limit to top 5
top_parent_asins = top_parent_asins.limit(5)

# Show the results
top_parent_asins.toPandas()


,parent_asin,average_rating,total_helpful_votes,rating_count
0,B085CFKBCK,5.0,11,5
1,B08R3CTLRY,5.0,4,2
2,B08R39X39B,5.0,1,1
3,B08R3BZKQ7,5.0,1,1
4,B08R39RB23,5.0,1,1


In [ ]:
# Define the structure for the data frame
schema = StructType([
    StructField('main_category', StringType(), True),
    StructField('title', StringType(), True),
    StructField('average_rating', FloatType(), True),
    StructField('rating_number', IntegerType(), True),
    StructField('features', StringType(), True),
    StructField('description', StringType(), True),
    StructField('price', FloatType(), True),
    StructField('images', StringType(), True),
    StructField('videos', StringType(), True),
    StructField('store', StringType(), True),
    StructField('categories', StringType(), True),
    StructField('details', StringType(), True),
    StructField('parent_asin', StringType(), True),
    StructField('bought_together', StringType(), True),
    StructField('subtitle', StringType(), True),
    StructField('author', StringType(), True)
])

df_meta = spark\
    .read\
    .schema(schema)\
    .json("meta_Amazon_Fashion.jsonl")\
    .repartition(20)
df_meta.rdd.getNumPartitions()
df_meta.count()

826108

In [ ]:
# Join 'top_parent_asins' with 'meta_df' on 'parent_asin'
top_parent_asin_details = top_parent_asins.join(
    df_meta,
    on="parent_asin",
    how="inner"
)

# Select only the desired columns from 'meta_df' and exclude average_rating, rating_count, total_helpful_votes from 'top_parent_asins'
desired_columns = [
    "parent_asin", "title", "features", "description",
    "price", "store", "categories", "details",
    "bought_together", "subtitle"
]

# Select these columns from the joined DataFrame
top_parent_asin_details = top_parent_asin_details.select(*desired_columns)

top_parent_asin_details.toPandas()

,parent_asin,title,features,description,price,store,categories,details,bought_together,subtitle
0,B08R3CTLRY,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Green, L)","[""Drawstring closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Date First Available"":""February 5, 2021""}",None,None
1,B085CFKBCK,DouBCQ Women's Summer Palazzo Comfy Lounge Flowy Wide Leg Casual Beach Pants(Brown Floral Black L),"[""Drawstring closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Item model number"":""Drop Crotch"",""Date First Available"":""May 8, 2020""}",None,None
2,B08R39X39B,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Flower White, XL)","[""Tie closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Item model number"":""Drop Crotch"",""Date First Available"":""February 5, 2021""}",None,None
3,B08R3BZKQ7,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Flower White,L)","[""Drawstring closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Item model number"":""Drop Crotch"",""Date First Available"":""February 5, 2021""}",None,None
4,B08R39RB23,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Camouflage, XL)","[""Drawstring closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Item model number"":""Drop Crotch"",""Date First Available"":""February 5, 2021""}",None,None


In [ ]:
# show the original item
filtered_df_meta = df_meta.filter(df_meta.parent_asin == "B08R39MRDW").select(*desired_columns)
filtered_df_meta.toPandas()

,parent_asin,title,features,description,price,store,categories,details,bought_together,subtitle
0,B08R39MRDW,"DouBCQ Women's Palazzo Lounge Wide Leg Casual Flowy Pants(Flower Mix Blue, XL)","[""Drawstring closure"",""Machine Wash""]",[],NaN,DouBCQ,[],"{""Package Dimensions"":""15 x 10.2 x 0.4 inches; 9.59 Ounces"",""Item model number"":""Drop Crotch"",""Date First Available"":""February 5, 2021""}",None,None
